In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras import layers

In [2]:
os.environ["KERAS_BACKEND"] = "jax"

**Grabbing the dataset:**

In [3]:
df = pd.read_csv("jena_climate_2009_2016.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420451 entries, 0 to 420450
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date Time        420451 non-null  object 
 1   p (mbar)         420451 non-null  float64
 2   T (degC)         420451 non-null  float64
 3   Tpot (K)         420451 non-null  float64
 4   Tdew (degC)      420451 non-null  float64
 5   rh (%)           420451 non-null  float64
 6   VPmax (mbar)     420451 non-null  float64
 7   VPact (mbar)     420451 non-null  float64
 8   VPdef (mbar)     420451 non-null  float64
 9   sh (g/kg)        420451 non-null  float64
 10  H2OC (mmol/mol)  420451 non-null  float64
 11  rho (g/m**3)     420451 non-null  float64
 12  wv (m/s)         420451 non-null  float64
 13  max. wv (m/s)    420451 non-null  float64
 14  wd (deg)         420451 non-null  float64
dtypes: float64(14), object(1)
memory usage: 48.1+ MB


In [5]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


**Preparing the dataset:**

In [6]:
num_train_samples = int(0.5 * len(df))
num_val_samples = int(0.25 * len(df))
num_test_samples = len(df) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

num_train_samples: 210225
num_val_samples: 105112
num_test_samples: 105114


In [7]:
dfRaw = df.drop(columns=["Date Time"])

In [8]:
mean = dfRaw[:num_train_samples].mean(axis=0)
dfRaw -= mean
std = dfRaw[:num_train_samples].std(axis=0)
dfRaw /= std

In [9]:
int_sequence = np.arange(10)
dummy_dataset = keras.utils.timeseries_dataset_from_array(
    data=int_sequence[:-3],
    targets=int_sequence[3:],
    sequence_length=3,
    batch_size=2,
)

for inputs, targets in dummy_dataset:
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))

[0, 1, 2] 3
[1, 2, 3] 4
[2, 3, 4] 5
[3, 4, 5] 6
[4, 5, 6] 7


In [10]:
sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256

train_dataset = keras.utils.timeseries_dataset_from_array(
    dfRaw[:-delay],
    targets=dfRaw["T (degC)"][delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples,
)

val_dataset = keras.utils.timeseries_dataset_from_array(
    dfRaw[:-delay],
    targets=dfRaw["T (degC)"][delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples,
)

test_dataset = keras.utils.timeseries_dataset_from_array(
    dfRaw[:-delay],
    targets=dfRaw["T (degC)"][delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples,
)

In [11]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

samples shape: (256, 120, 14)
targets shape: (256,)


## Modelling:

In [12]:
type(dfRaw)

pandas.core.frame.DataFrame

**Using layer_lstm() instead of layer_gru()**

In [13]:
inputs = keras.Input(shape=(sequence_length, dfRaw.shape[-1]))
x = layers.LSTM(32)(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "jena_lstm_dropout.keras", save_best_only=True
    )
]
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [15]:
histLSTM = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=callbacks,
)

Epoch 1/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - loss: 0.1657 - mae: 0.3171 - val_loss: 0.1137 - val_mae: 0.2615
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - loss: 0.1559 - mae: 0.3077 - val_loss: 0.1128 - val_mae: 0.2620
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 33s 40ms/step - loss: 0.1475 - mae: 0.2996 - val_loss: 0.1117 - val_mae: 0.2602
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 32s 39ms/step - loss: 0.1434 - mae: 0.2953 - val_loss: 0.1104 - val_mae: 0.2582
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 45s 55ms/step - loss: 0.1387 - mae: 0.2906 - val_loss: 0.1168 - val_mae: 0.2653
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 42s 51ms/step - loss: 0.1339 - mae: 0.2857 - val_loss: 0.1192 - val_mae: 0.2692
Epoch 7/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 32s 39ms/step - loss: 0.1309 - mae: 0.2827 - val_loss: 0.1223 - val_mae: 0.2732
Epoch 8/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - loss: 0.1263 - mae: 0.2781 - val_loss: 0.1281 - val_mae: 0.2803
Epoch 9/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 

**Adjusting the number of units in each recurrent layer in the stacked setup:**

In [17]:
inputs = keras.Input(shape=(sequence_length, dfRaw.shape[-1]))
x = layers.GRU(16, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(32, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
modSTACK = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "jena_stacked_gru_dropout.keras", save_best_only=True
    )
]
modSTACK.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [18]:
histSTACK = modSTACK.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=callbacks,
)

Epoch 1/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 51s 60ms/step - loss: 0.2097 - mae: 0.3531 - val_loss: 0.1174 - val_mae: 0.2669
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 51s 62ms/step - loss: 0.1665 - mae: 0.3182 - val_loss: 0.1150 - val_mae: 0.2642
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 47s 58ms/step - loss: 0.1622 - mae: 0.3140 - val_loss: 0.1136 - val_mae: 0.2622
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 49s 59ms/step - loss: 0.1592 - mae: 0.3108 - val_loss: 0.1130 - val_mae: 0.2616
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 49s 60ms/step - loss: 0.1568 - mae: 0.3081 - val_loss: 0.1119 - val_mae: 0.2602
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 59s 72ms/step - loss: 0.1547 - mae: 0.3060 - val_loss: 0.1116 - val_mae: 0.2599
Epoch 7/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 67s 81ms/step - loss: 0.1524 - mae: 0.3040 - val_loss: 0.1101 - val_mae: 0.2575
Epoch 8/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 67s 82ms/step - loss: 0.1491 - mae: 0.3005 - val_loss: 0.1099 - val_mae: 0.2579
Epoch 9/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 

### Test Set Evaluation:

**LSTM:**

In [19]:
modLSTM = keras.models.load_model("jena_lstm_dropout.keras")
print(f"Test MAE: {modLSTM.evaluate(test_dataset)[1]:.2f}")

405/405 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 0.1269 - mae: 0.2791
Test MAE: 0.28


**STACKED:**

In [20]:
modSTACK = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {modSTACK.evaluate(test_dataset)[1]:.2f}")

405/405 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - loss: 0.1246 - mae: 0.2743
Test MAE: 0.27
